Youtube Video Summariser

In [3]:
! pip install pytube -q

In [6]:
from pytube import YouTube
import subprocess

In [98]:
URL  = 'https://www.youtube.com/watch?v=wD2cVhC-63I'

Create a YouTube object and get the audio stream

In [58]:
yt = YouTube(URL)
audio_stream = yt.streams.filter(only_audio=True, file_extension='mp4').first()

Download the audio stream as an MP4 file

In [59]:
audio_stream.download(filename='audio.mp4')


'/content/audio.mp4'

Use FFmpeg to convert the downloaded MP4 audio to WAV

In [60]:
! ffmpeg -i ytaudio.mp4 -acodec pcm_s16le -ar 16000 ytaudio.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [61]:
! ffmpeg -version

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

In [62]:
!pip install huggingsound -q

Importing modules for the converting audio to text

In [63]:
from huggingsound import SpeechRecognitionModel
import torch

In [64]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [65]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)

INFO:huggingsound.speech_recognition.model:Loading model...


Audio Chunking - Breaking down audio into chunks for speech to text conversion for the limited GPU

In [66]:
import librosa

In [67]:
input_file = '/content/ytaudio.wav'

In [68]:
print(librosa.get_samplerate(input_file))

# Stream over 30 seconds chunks rather than load the full file
stream = librosa.stream(
    input_file,
    block_length=30,
    frame_length=16000,
    hop_length=16000
)

16000


In [69]:
import soundfile as sf

In [70]:
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav', speech, 16000)

Audio to Text

In [71]:
audio_path =[]
for a in range(i+1):
  audio_path.append(f'/content/{a}.wav')

In [72]:
audio_path

['/content/0.wav',
 '/content/1.wav',
 '/content/2.wav',
 '/content/3.wav',
 '/content/4.wav',
 '/content/5.wav',
 '/content/6.wav',
 '/content/7.wav',
 '/content/8.wav',
 '/content/9.wav']

In [75]:
transcriptions = model.transcribe(audio_path)

100%|██████████| 10/10 [05:34<00:00, 33.48s/it]


In [81]:
full_transcript = ' '
for item in transcriptions:
  full_transcript += ''.join(item['transcription'])
print(full_transcript)

 arwhat was is funny its such in honor to present this next award the fivemen i'm still watching you the fve-men nominated in this category delivered resonating evocative and captivating performances here are the nominees for performance by an actor in a leading rolechristian bale american huzzel bruce dern nebraska leonardo dicabrio the wolfrall street chewatl giophor twelve years lafe matthew mcconaghey dallas byers rodand the oscar goes tomathew mconoghey-thankyouthank you thak you tu to the academy for alsix thousenmembers thank you nominee all these performances were impeccable in my opinion no didn't see a false note anywherei want to think chellmark valet are director ontik cherletto genfer garner who worked with daileythets a few things  three things to my account that i need each dayneto look up to another is somethin to look forward to and another is someone to chaseothank godbcause at'swhi look up tohes graced my life with opportunities that i know or ot of my hand eny other

OPEN AI API Summary

In [43]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.0 MB/s eta 0:00:00


In [92]:
import openai
import json

In [93]:
API_KEY = 'API_KEY'

In [94]:
openai.api_key = API_KEY

In [95]:
ask = "This is Youtube Transcript generated from Hugging Face ASR and Now this text need to be summarised. Please give a brief summary"

In [97]:
response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role":"user", "content":ask+full_transcript}
    ]
)

content = response['choices'][0]['message']['content']
print(content)

The presenter announces the nominees for the Best Actor category at the Oscars, and Matthew McConaughey wins the award. In his acceptance speech, he thanks the Academy and acknowledges the impeccable performances of his fellow nominees. He expresses gratitude to his director, family, and wife for their support. McConaughey also shares a story about his personal hero and explains that he always sees his hero as someone ten years ahead of him, providing him with someone to chase after. He concludes by saying, "alright, alright, alright," referencing his famous catchphrase.
